## importing libraries

In [1]:
import os
from langchain.agents import *
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import AgentExecutor
from dotenv import load_dotenv
import os
load_dotenv()

True

### Database connection

In [4]:
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

### let's setup the LLMs, toolkit, and agent executer


In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI 
import google.generativeai as genai


KEY=os.getenv("GOOGLE_API_KEY")

# genai.GenerativeModel('gemini-pro')
genai.configure(api_key=KEY)

In [18]:
from langchain.schema.messages import HumanMessage

model = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)
model.invoke("what is data?").content

'Data is a collection of facts, figures, and other information that has been organized and processed so that it can be used for analysis, decision-making, or other purposes. Data can be numerical, textual, or graphical, and it can be collected from a variety of sources, such as surveys, questionnaires, experiments, and observations.\n\nData is essential for understanding the world around us. It helps us to make informed decisions, to identify trends, and to predict future outcomes. Data can also be used to test hypotheses, to evaluate the effectiveness of programs, and to communicate information to others.\n\nThere are many different types of data, each with its own unique characteristics. Some common types of data include:\n\n* **Numerical data:** Data that can be represented by numbers, such as temperature, height, or weight.\n* **Textual data:** Data that consists of text, such as a name, address, or description.\n* **Graphical data:** Data that is represented in a visual format, su

In [19]:
toolkit = SQLDatabaseToolkit(db=db, llm=model)

In [20]:
agent_executor = create_sql_agent(
    llm=model,
    toolkit=toolkit,
    verbose=True
)

### let's check


In [22]:
agent_executor.run("what is total sales month wise?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: abc, orders, orders_details, pizza_types, pizzasAction: sql_db_query_checker
Action Input: select substr(order_date, 1, 7) AS Month, sum(quantity*price) as total_sales from orders_details od join orders o on o.order_id = od.order_id group by substr(order_date, 1, 7)```sql
SELECT SUBSTRING(order_date, 1, 7) AS Month, SUM(quantity * price) AS total_sales 
FROM orders_details od 
JOIN orders o ON o.order_id = od.order_id 
GROUP BY SUBSTRING(order_date, 1, 7);
```Action: sql_db_query
Action Input: select substr(order_date, 1, 7) AS Month, sum(quantity*price) as total_sales from orders_details od join orders o on o.order_id = od.order_id group by substr(order_date, 1, 7)Error: (pymysql.err.OperationalError) (1054, "Unknown column 'order_date' in 'field list'")
[SQL: select substr(order_date, 1, 7) AS Month, sum(quantity*price) as total_sales from orders_details od join orders o on o.order_id = od.order_id 

"[('2015-02', 77980.10000000033), ('2015-01', 83121.8400000004), ('2015-04', 82136.72000000032), ('2015-03', 84907.62000000052), ('2015-06', 81564.00000000083), ('2015-05', 85008.37000000042), ('2015-08', 82121.97000000102), ('2015-07', 86689.44000000042), ('2015-10', 76134.16000000054), ('2015-09', 77238.05000000041), ('2015-11', 84711.3300000005), ('2015-12', 77573.45000000035)]"

In [28]:
agent_executor.run("I want all pizza price")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: abc, orders, orders_details, pizza_types, pizzasAction: sql_db_schema
Action Input: pizzas
CREATE TABLE pizzas (
	pizza_id TEXT, 
	pizza_type_id TEXT, 
	size TEXT, 
	price DOUBLE
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from pizzas table:
pizza_id	pizza_type_id	size	price
bbq_ckn_s	bbq_ckn	S	12.7500000000
bbq_ckn_m	bbq_ckn	M	16.7500000000
bbq_ckn_l	bbq_ckn	L	20.7500000000
*/Action: sql_db_query
Action Input: SELECT price FROM pizzas;[(12.75,), (16.75,), (20.75,), (12.75,), (16.75,), (20.75,), (12.75,), (16.75,), (20.75,), (12.75,), (16.75,), (20.75,), (12.75,), (16.75,), (20.75,), (12.75,), (16.75,), (20.75,), (12.0,), (16.0,), (20.5,), (12.0,), (16.0,), (20.5,), (10.5,), (13.25,), (16.5,), (12.0,), (16.0,), (20.5,), (12.0,), (16.0,), (20.5,), (11.0,), (14.5,), (17.5,), (9.75,), (12.5,), (15.25,), (12.0,), (16.0,), (20.5,), (25.5,), (35.95,), (23.65,), (12.25,), (16

'12.75, 16.75, 20.75, 12.75, 16.75, 20.75, 12.75, 16.75, 20.75, 12.75, 16.75, 20.75, 12.75, 16.75, 20.75, 12.75, 16.75, 20.75, 12.0, 16.0, 20.5, 12.0, 16.0, 20.5, 10.5, 13.25, 16.5, 12.0, 16.0, 20.5, 12.0, 16.0, 20.5, 11.0, 14.5, 17.5, 9.75, 12.5, 15.25, 12.0, 16.0, 20.5, 25.5, 35.95, 23.65, 12.25, 16.25, 20.25, 12.5, 16.5, 20.75, 12.5, 16.5, 20.75, 12.5, 16.5, 20.75, 12.25, 16.25, 20.25, 12.5, 16.5, 20.75, 12.5, 16.5, 20.75, 12.5, 16.5, 20.75, 12.5, 15.5, 18.5, 11.75, 14.75, 17.95, 12.0, 16.0, 20.25, 12.75, 16.75, 21.0, 12.0, 16.0, 20.25, 12.0, 16.0, 20.25, 12.5, 16.5, 20.75, 12.0, 16.0, 20.25, 12.0, 16.0, 20.25'